In [1]:
# DATA EXTRACTION - Population

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage containing the table
url = 'https://cuentame.inegi.org.mx/monografias/informacion/mex/poblacion/default.aspx?tema=me&e=15'

# Send a GET request to the webpage
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.text, 'html.parser')

# Find the nested table within the outer table by its id
inner_table = soup.find('table', id='keywords2')  # Adjust id name as needed

# Check if inner table is found
if inner_table:
    # Initialize lists to store the data
    data = []

    # Loop through each row in the inner table
    for row in inner_table.find_all('tr'):
        # Initialize list to store data for each row
        row_data = []
        # Loop through each cell in the row
        for cell in row.find_all('td'):
            # Append the cell content to the row_data list
            row_data.append(cell.text.strip())
        # Append the row_data list to the data list
        data.append(row_data)

    # Convert the data list to a DataFrame
    df = pd.DataFrame(data)
    print(df)

        0                               1        2
0    None                            None     None
1     001    Acambay    de Ruíz Castañeda   67,872
2     002                         Acolman  171,507
3     003                          Aculco   49,266
4     004       Almoloya    de Alquisiras   15,333
..    ...                             ...      ...
121   121           Cuautitlán    Izcalli  555,163
122   122  Valle de    Chalco Solidaridad  391,731
123   123                        Luvianos   28,822
124   124          San José    del Rincón  100,082
125   125                       Tonanitla   14,883

[126 rows x 3 columns]


In [2]:
# DATA WRANGLER - Population

# Drop the top an bottom rows
df = df.drop(df.index[0], axis=0)
df = df.drop(df.columns[0], axis=1)

# Remove commas and convert to float
df.iloc[:, 1] = df.iloc[:, 1].str.replace(',', '').astype(int)

# Rename the columns
new_column_names = ['Municipality', 'Population']
df.columns = new_column_names

# Delete extra spaces
df = df.replace(r'\s+', ' ', regex=True)

# Delete "San Simón de Guerrero" because of missing data in the cases table
df = df[df['Municipality'] != 'San Simón de Guerrero']

populationByMunicipality = df

In [3]:
populationByMunicipality

,Municipality,Population
1,Acambay de Ruíz Castañeda,67872
2,Acolman,171507
3,Aculco,49266
4,Almoloya de Alquisiras,15333
5,Almoloya de Juárez,174587
...,...,...
121,Cuautitlán Izcalli,555163
122,Valle de Chalco Solidaridad,391731
123,Luvianos,28822
124,San José del Rincón,100082


In [4]:
# DATA EXTRACTION - COVID Cases

# URL of the webpage containing the table
url = 'https://salud.edomex.gob.mx/salud/covid19_municipio'

# Send a GET request to the webpage
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table by its class
table = soup.find('table', class_='table table-striped tablaestadistica')  # Replace 'your-table-class' with the actual class name

# Initialize lists to store the data
data = []

# Loop through each row in the table
for row in table.find_all('tr'):
    # Initialize list to store data for each row
    row_data = []
    # Loop through each cell in the row
    for cell in row.find_all(['th', 'td']):  # Include both th and td elements
        # Append the cell content to the row_data list
        row_data.append(cell.text.strip())
    # Append the row_data list to the data list
    data.append(row_data)

# Convert the data list to a DataFrame
df = pd.DataFrame(data)

In [5]:
# DATA WRANGLING - COVID Cases

df = df.iloc[1:-1]

# Rename columns
new_column_names = ['Municipality', 'Cases', 'Deaths']
df.columns = new_column_names

# Delete "San Simón de Guerrero" because of missing data in the cases table
df = df[df['Municipality'] != 'San Simón de Guerrero']

import numpy as np

# Remove commas and convert to integer for the second and third columns
df.iloc[:, 1:3] = df.iloc[:, 1:3].replace(',', '', regex=True).astype(int)

covidCasesByMunicipality = df

In [6]:
covidCasesByMunicipality

,Municipality,Cases,Deaths
1,Ecatepec de Morelos,23799,3141
2,Nezahualcóyotl,18330,1081
3,Toluca,14387,2373
4,Naucalpan,13338,1315
5,Tlalnepantla,11358,1059
...,...,...,...
121,Papalotla,29,4
122,Ecatzingo,27,1
123,Zacazonapan,18,1
124,Ixtapan del Oro,15,1


In [7]:
# DATA NORMALIZATION

# Dictionary mapping old names to new names
old_names_to_new_names = {
    'Teoloyucan': 'Teoloyucán',
}

# Replace the old names with the new names
populationByMunicipality['Municipality'] = populationByMunicipality['Municipality'].replace(old_names_to_new_names)

old_names_to_new_names = {
    'Acambay': 'Acambay de Ruíz Castañeda',
    'Valle de Chalco': 'Valle de Chalco Solidaridad',
    'Naucalpan': 'Naucalpan de Juárez',
    'Tlalnepantla': 'Tlalnepantla de Baz',
}

# Replace the old names with the new names
covidCasesByMunicipality['Municipality'] = covidCasesByMunicipality['Municipality'].replace(old_names_to_new_names)


# Get the set of unique municipalities in each DataFrame
municipalities_df1 = set(populationByMunicipality['Municipality'])
municipalities_df2 = set(covidCasesByMunicipality['Municipality'])

# Find the municipalities that exist in both DataFrames
common_municipalities = municipalities_df1.intersection(municipalities_df2)

# Print the municipalities that exist in both DataFrames
print("Municipalities that exist in both DataFrames:")
print(common_municipalities)

# Find the municipalities that exist in one DataFrame but not in the other
unique_to_df1 = municipalities_df1 - municipalities_df2
unique_to_df2 = municipalities_df2 - municipalities_df1

# Print the municipalities that exist in one DataFrame but not in the other
print("Municipalities unique to DataFrame 1:")
print(unique_to_df1)
print("Municipalities unique to DataFrame 2:")
print(unique_to_df2)


Municipalities that exist in both DataFrames:
{'Jocotitlán', 'Zinacantepec', 'Tepotzotlán', 'Ixtapan del Oro', 'Atenco', 'Temascaltepec', 'Tonanitla', 'Temoaya', 'Nextlalpan', 'Nopaltepec', 'Amanalco', 'Lerma', 'Tecámac', 'Hueypoxtla', 'Toluca', 'Xalatlaco', 'Amatepec', 'Ayapango', 'Mexicaltzingo', 'Apaxco', 'Villa de Allende', 'Amecameca', 'Tultepec', 'Nicolás Romero', 'Cuautitlán Izcalli', 'Polotitlán', 'Tonatico', 'Jaltenco', 'Chapultepec', 'Tianguistenco', 'Atizapán de Zaragoza', 'Ixtapaluca', 'Isidro Fabela', 'Texcoco', 'Atlautla', 'Metepec', 'Almoloya del Río', 'Aculco', 'Capulhuac', 'Ixtapan de la Sal', 'San José del Rincón', 'San Antonio la Isla', 'Morelos', 'Zumpahuacán', 'Jiquipilco', 'Tejupilco', 'Rayón', 'San Felipe del Progreso', 'Villa Victoria', 'Temamatla', 'Tenango del Aire', 'Melchor Ocampo', 'Otumba', 'Chiautla', 'El Oro', 'Jilotepec', 'Valle de Chalco Solidaridad', 'Atlacomulco', 'Temascalapa', 'Huixquilucan', 'Temascalcingo', 'Chicoloapan', 'Valle de Bravo', 'Chapa

In [8]:
# DATA INTEGRATION

# Merge both DF based on the column Municipality
merged_df = populationByMunicipality.merge(covidCasesByMunicipality, on='Municipality')

In [9]:
merged_df

,Municipality,Population,Cases,Deaths
0,Acambay de Ruíz Castañeda,67872,440,39
1,Acolman,171507,1249,103
2,Aculco,49266,113,8
3,Almoloya de Alquisiras,15333,67,7
4,Almoloya de Juárez,174587,922,180
...,...,...,...,...
119,Cuautitlán Izcalli,555163,5257,1046
120,Valle de Chalco Solidaridad,391731,2181,412
121,Luvianos,28822,170,36
122,San José del Rincón,100082,218,35


In [68]:
merged_df.to_csv('casesCovid.csv', index=False)